<a href="https://colab.research.google.com/github/mitosagi/puzzdra-nnsolver/blob/master/puzz_move.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 初期化

In [23]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [24]:
!cp -r /content/drive/MyDrive/User/python/puzzdra-nnsolver /content/puzzdra-nnsolver
%cd /content/puzzdra-nnsolver
!pip install --log=pip_log -e .
!pip install git+https://github.com/DLR-RM/stable-baselines3

/content/puzzdra-nnsolver
Obtaining file:///content/puzzdra-nnsolver
  Preparing metadata (setup.py) ... done
  Attempting uninstall: Puzzpy
    Found existing installation: Puzzpy 1.0
    Uninstalling Puzzpy-1.0:
      Successfully uninstalled Puzzpy-1.0
  Running setup.py develop for Puzzpy
  Cloning https://github.com/DLR-RM/stable-baselines3 to /tmp/pip-req-build-2eddgafv
  Running command git clone --filter=blob:none --quiet https://github.com/DLR-RM/stable-baselines3 /tmp/pip-req-build-2eddgafv
  Resolved https://github.com/DLR-RM/stable-baselines3 to commit 69afefc91d408d352b4224ae5244ad2c32bb7634
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for stable-baselines3: filename=stable_baselines3-2.2.0a9-py3-none-any.whl size=180895 sha256=021b8a108e8de49c5156f9e25f5a71192b8b8578ee4fd50d6ca39f296c20a1c9
  Stored in directory: /tmp/pip-ephem-wheel-cache-eiih2sxx/wheels/3b/24/65/bc2

## 実際の処理

In [151]:
import numpy as np
import gymnasium
from gymnasium import spaces
from puzzpy import PuzzTable
from stable_baselines3.common.env_checker import check_env
import random

drop_color = 3
board_width = 6
board_height = 5
min_size = 36

class bcolors:
    HEADER = '\033[95m'
    OKBLUE = '\033[94m'
    OKCYAN = '\033[96m'
    OKGREEN = '\033[92m'
    WARNING = '\033[93m'
    FAIL = '\033[91m'
    ENDC = '\033[0m'
    BOLD = '\033[1m'
    UNDERLINE = '\033[4m'

class PuzzEnv(gymnasium.Env):
  """
  パズドラの環境
  """
  # ColabのためGUIを実装できない
  metadata = {'render.modes': ['console']}

  def __init__(self):
    super(PuzzEnv, self).__init__()

    self.action_space = spaces.Discrete(4) # 指を離す動作を含めると5
    self.observation_space = spaces.Box(low=0, high=255, shape=(1+1+drop_color, max(min_size, board_height), max(min_size, board_width)), dtype=np.uint8) # 1 for turn num, +1 for finger position
    self.prev_action = 255
    self.prev_finger_array = None

  def retobs(self, table):
    turn_array = [np.full((board_height, board_width), table.get_turn(), np.uint8)]
    finger_array = np.array([np.array(table.get_XY_as_table()).astype(np.uint8) * (2 ** 5)]).astype(np.uint8)
    if self.prev_finger_array is not None:
      finger_array = finger_array + self.prev_finger_array // 2
    self.prev_finger_array = finger_array
    table_array = np.array(table.get_table()).astype(np.uint8)
    u = np.arange(table_array.max()+1)
    table_array = (u[:,np.newaxis,np.newaxis]==table_array).astype(np.uint8)[1:] # ドロップをone-hotにする https://stackoverflow.com/questions/67249470/convert-a-2d-numpy-array-into-a-hot-encoded-3d-numpy-array-with-same-values-in
    tmp_array = np.concatenate([turn_array, finger_array, table_array])
    rew = table.eval_otoshi()
    return np.pad(tmp_array, [(0,0), (0, abs(min_size - board_height)), (0, abs(min_size - board_width))], mode='constant', constant_values=0), rew, True, False, {}

  def reset(self, seed = None, test_min = 30):
    test_min = random.randrange(10,30,1)
    super().reset(seed=seed)
    """
    【重要】観測はnumpy配列でなければならない
    :return: (np.array)
    """
    while True:
      self.table = PuzzTable("".join([str(random.randrange(drop_color)) for i in range(board_width*board_height)]), random.randrange(board_width), random.randrange(board_height), test_min) # n色陣　操作時間m秒
      if self.table.eval_otoshi() == 0:
        break

    return self.retobs(self.table)[0], {}

  def step(self, action):
    if action == 4: # 指を離す動作
      return self.retobs(self.table)

    next_table = self.table.next_tables()[action]

    if abs(action - self.prev_action) == 2: # 元の方向に戻る 0と2 1と3が該当
      # next_table = self.table
      return self.retobs(self.table)[0], -1, True, False, {}
    self.prev_action = action

    if next_table.get_table()[0][0] == 127: # 壁に移動
      # next_table = self.table
      return self.retobs(self.table)[0], -1, True, False, {}

    self.table = next_table

    if self.table.get_turn() <= 0: # 時間切れ
      return self.retobs(self.table)

    return self.retobs(self.table)[0], 0, False, False, {}

  def render(self, mode='console', close=False):
    if mode != 'console':
      raise NotImplementedError()

    start = self.table.get_XY_as_table()
    table = self.table.get_table()
    for i in range(board_height):
      for j in range(board_width):
        if start[i][j] == 1:
          print(bcolors.FAIL + str(table[i][j]) + bcolors.ENDC, end='')
        else:
          print(table[i][j], end='')

      print('')

check_env(PuzzEnv())

In [152]:
env = PuzzEnv()

obs = env.reset()
env.render()

print(env.observation_space)
print(env.action_space)
print(env.action_space.sample())

n_steps = 20
for step in range(n_steps):
  print("Step {}".format(step + 1))
  obs, reward, done, _, info = env.step(env.action_space.sample())
  print('obs=', obs.shape, 'reward=', reward, 'done=', done)
  with np.printoptions(threshold=np.inf):
    print(obs)
  env.render()
  if done:
    print("Goal reached!", "reward=", reward)
    break

331323
213132
321223
223121
332233
Box(0, 255, (5, 36, 36), uint8)
Discrete(4)
2
Step 1
obs= (5, 36, 36) reward= 0 done= False
[[[27 27 27 27 27 27  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
    0  0  0  0  0  0  0  0  0  0  0  0  0]
  [27 27 27 27 27 27  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
    0  0  0  0  0  0  0  0  0  0  0  0  0]
  [27 27 27 27 27 27  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
    0  0  0  0  0  0  0  0  0  0  0  0  0]
  [27 27 27 27 27 27  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
    0  0  0  0  0  0  0  0  0  0  0  0  0]
  [27 27 27 27 27 27  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
    0  0  0  0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
    0  0  0  0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
    0  0  0  0  0  0  0  0  0  0  0  0  0]
  [ 0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0

In [149]:
%load_ext tensorboard
%reload_ext tensorboard
# %tensorboard --logdir puzzdra_tensorboard
%tensorboard --logdir puzzdra_tensorboard --host=127.0.0.1 --port=6006 --load_fast=false
from google.colab import output
output.serve_kernel_port_as_window(6006, path="")

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


<IPython.core.display.Javascript object>

/usr/local/lib/python3.10/dist-packages/google/colab/output/_util.py:114: DeprecationWarning: This has been deprecated due to changes in browser security. Use `serve_kernel_port_as_iframe` instead.
  warnings.warn(


<IPython.core.display.Javascript object>

In [150]:
# 実行前にログ名を設定すること!!!
log_name='PPO gamma no turn no prev'

from stable_baselines3 import PPO, SAC
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3.common.envs.multi_input_envs import SimpleMultiObsEnv
from stable_baselines3.common.callbacks import BaseCallback
from statistics import mean

log_name='PPO gamma'

# 環境の生成
env = PuzzEnv()

# 環境のラップ
monienv = Monitor(env, filename=None, allow_early_resets=True)

# エージェントの訓練
model = PPO('CnnPolicy', monienv, verbose=1, tensorboard_log="./puzzdra_tensorboard/")

class TensorboardCallback(BaseCallback):
  """
  Custom callback for plotting additional values in tensorboard.
  """

  def __init__(self, verbose=0):
    super(TensorboardCallback, self).__init__(verbose)
    self.check_freq = 10000

  def _on_step(self) -> bool:
    if self.n_calls % self.check_freq == 0:

      env = PuzzEnv()
      rew_array = []

      for test in range(100):
        obs, _info = env.reset()
        n_steps = 100
        for step in range(n_steps):
          action, _ = model.predict(obs, deterministic=True)
          obs, reward, done, _, info = env.step(action)
          if done:
            rew_array.append(reward)
            break

      self.logger.record('combo', mean(rew_array) if len(rew_array) > 0 else 0)
    return True

model = model.learn(50000, tb_log_name=log_name, callback=TensorboardCallback()) # 1500000 = 1 hour

Using cuda device
Wrapping the env in a DummyVecEnv.
Logging to ./puzzdra_tensorboard/PPO gamma_4
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 29.5     |
|    ep_rew_mean     | 1.26     |
| time/              |          |
|    fps             | 337      |
|    iterations      | 1        |
|    time_elapsed    | 6        |
|    total_timesteps | 2048     |
---------------------------------
-----------------------------------------
| rollout/                |             |
|    ep_len_mean          | 28.9        |
|    ep_rew_mean          | 1.4         |
| time/                   |             |
|    fps                  | 289         |
|    iterations           | 2           |
|    time_elapsed         | 14          |
|    total_timesteps      | 4096        |
| train/                  |             |
|    approx_kl            | 0.009283457 |
|    clip_fraction        | 0.0165      |
|    clip_range           | 0.2         |
|    entropy_los

In [60]:
#
env = PuzzEnv()

obs, _info = env.reset()
env.render()

print(env.observation_space)
print(env.action_space)
print(env.action_space.sample())

n_steps = 20
for step in range(n_steps):
  print("Step {}".format(step + 1))
  action, _states = model.predict(obs)
  obs, reward, done, _, info = env.step(action)
  # print('obs=', obs, 'reward=', reward, 'done=', done)
  env.render()
  if done:
    print("Goal reached!", "reward=", reward)
    break

321321
212312
233131
321213
132312
Box(0, 255, (1, 36, 36), uint8)
Discrete(4)
2
Step 1
321321
212312
233131
321213
132312
Goal reached! reward= -10


In [ ]:
model.save('puzzdra_nn')